In [ ]:
import cudf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from cuml.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("cleaned_dataset.csv")
X = df['cleaned_review'].values
y = df['sentiment'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)
        return self.fc(hn[-1])
    
class CNNModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_dim, 100, kernel_size=3)
        self.fc = nn.Linear(100, output_dim)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool1d(x, x.size(2)).squeeze(2)
        return self.fc(x)
    
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, hn = self.rnn(x)
        return self.fc(hn[-1])
    
class LSTMCNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMCNNModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.conv = nn.Conv1d(hidden_dim, 100, kernel_size=3)
        self.fc = nn.Linear(100, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out.permute(0, 2, 1)  # Prepare for Conv1d
        conv_out = self.conv(lstm_out)
        conv_out = nn.functional.relu(conv_out)
        conv_out = nn.functional.max_pool1d(conv_out, conv_out.size(2)).squeeze(2)
        return self.fc(conv_out)

In [ ]:
def train_model(model, train_loader, epochs=5):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()

            # Backward and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')
    return model

In [ ]:
input_dim = X_train_tensor.size(1)
output_dim = len(np.unique(y_train))
hidden_dim = 128

lstm_model = LSTMModel(input_dim, hidden_dim, output_dim)
cnn_model = CNNModel(input_dim, output_dim)
rnn_model = RNNModel(input_dim, hidden_dim, output_dim)
lstm_cnn_model = LSTMCNNModel(input_dim, hidden_dim, output_dim)

In [ ]:
lstm_model = train_model(lstm_model, train_loader)
cnn_model = train_model(cnn_model, train_loader)
rnn_model = train_model(rnn_model, train_loader)
lstm_cnn_model = train_model(lstm_cnn_model, train_loader)

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    acc = accuracy_score(all_labels, all_preds)
    print("Accuracy:", acc)
    print(classification_report(all_labels, all_preds))
    sns.heatmap(confusion_matrix(all_labels, all_preds), annot=True, fmt="d")
    plt.show()

In [ ]:
print("LSTM Model:")
evaluate_model(lstm_model, test_loader)

In [ ]:
print("CNN Model:")
evaluate_model(cnn_model, test_loader)

In [ ]:
print("RNN Model:")
evaluate_model(rnn_model, test_loader)

In [ ]:
print("LSTM-CNN Hybrid Model:")
evaluate_model(lstm_cnn_model, test_loader)